## Guardrails
Guardrails w LangChain to mechanizmy pilnujące, aby odpowiedzi modelu spełniały określone wymagania techniczne lub formatowe.
Na przykład JsonValidityEvaluator sprawdza, czy zwrócony tekst jest poprawnym JSON-em – jeśli struktura jest niepoprawna, evaluator zwróci błąd.
Dzięki temu można automatycznie wykrywać i odrzucać odpowiedzi, które nie nadają się do dalszego przetwarzania.

### JSON Format Validator
Sprawdza, czy wygenerowana odpowiedź jest poprawnym JSON-em

In [20]:
from langchain.evaluation import JsonValidityEvaluator

evaluator = JsonValidityEvaluator()
# print(evaluator.evaluate_strings(prediction='{"x": 1}'))      # poprawne
print(evaluator.evaluate_strings(prediction='{x: 1}'))        # błąd


{'score': 0, 'reasoning': 'Expecting property name enclosed in double quotes: line 1 column 2 (char 1)'}


### JsonEqualityEvaluator
Sprawdza równość JSON-ów po sparsowaniu (kolejność kluczy w JSON nie ma znaczenia)

In [23]:
from langchain.evaluation import JsonEqualityEvaluator

evaluator = JsonEqualityEvaluator()
print(evaluator.evaluate_strings(
    prediction='{"a":1,"b":[2,3]}',
    reference='{"b":[2,3],"a":2}',
))


{'score': False}


### RegexMatchEvaluator
Sprawdza dopasowanie do wyrażenia regularnego

In [26]:
from langchain.evaluation import RegexMatchStringEvaluator

evaluator = RegexMatchStringEvaluator()
result = evaluator.evaluate_strings(
    prediction="Order ID: ABC-1234",
    reference=r"^Order ID: [A-Z]{3}-\d{4}$",
)
print(result['score'])

iter = 3
while result['score'] < 1.0 and iter > 0:
    iter -= 1
    print('run model once more')

0
run model once more
run model once more
run model once more


### Fallback Messages Validator
Kiedy główny model zawiedzie (rate limit / błąd) automatycznie użyj modelu zapasowego.

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

primary = ChatOpenAI(model="gpt-4o-miniS", max_retries=0)  # wyłącz retry, by fallback zadziałał
backup  = ChatOpenAI(model="gpt-3.5-turbo")

chain = primary.with_fallbacks([backup])

print(chain.invoke("Opisz w 1 zdaniu Python."))


content='Python to wysokopoziomowy język programowania, znany ze swojej przejrzystej składni i wszechstronności, który jest powszechnie stosowany do tworzenia różnego rodzaju aplikacji i skryptów.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 16, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CPP4qEWR8cQ2EV3ArBAXyOtzxCjjh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--8b72b4ab-ca69-441b-8ef5-6b0cc441f2b2-0' usage_metadata={'input_tokens': 16, 'output_tokens': 59, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### Token Limit
Śledzenie i przycinanie historii do limitu tokenów, żeby nie przekroczyć kontekstu modelu.

In [29]:
import json
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.messages.utils import trim_messages, count_tokens_approximately
from langchain_openai import ChatOpenAI

messages = [
    SystemMessage(content="Jesteś pomocnym asystentem."),
    HumanMessage(content="(tu długa historia rozmowy / wiele wiadomości...)"),
]

trimmed = trim_messages(
    messages,
    strategy="last",
    token_counter=count_tokens_approximately,
    max_tokens=256,
    start_on="human",
    include_system=True,
)

llm = ChatOpenAI(model="gpt-4o-mini")
print(json.dumps(llm.invoke(trimmed).response_metadata, indent=4))


{
    "token_usage": {
        "completion_tokens": 36,
        "prompt_tokens": 33,
        "total_tokens": 69,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0
        }
    },
    "model_name": "gpt-4o-mini-2024-07-18",
    "system_fingerprint": "fp_560af6e559",
    "id": "chatcmpl-CPBcjgDnYifYhkEWXRH30M4AXrFpZ",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
}


### Word Limit
poproś o max N słów, po generacji policz słowa. W razie przekroczenia – skróć odpowiedź.

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

limit = 25
prompt = f"Napisz streszczenie w MAKS {limit} słowach: Czym jest uczenie maszynowe?"

resp = llm.invoke(prompt).content
if len(resp.split()) > limit:
    # szybka korekta – poproś model o skrócenie do limitu
    resp = llm.invoke(f"Skróć to do maks {limit} słów, bez dopisków:\n\n{resp}").content

print(resp)


Uczenie maszynowe to dziedzina sztucznej inteligencji, która umożliwia komputerom uczenie się z danych i doskonalenie swoich działań bez programowania.
